# Why AI Agent 

In this notebook we will build a simple AI Agent to answer the question `What day is today?`. We will:

* [Try to understand why LLM Hallucinate](#why-llm-hallucinate)
* [Create a simple tool in Python to help the LLM not to hallucinate](#create-a-simple-tool-in-python)
* [Define the tool for the LLM](#tool-definition-for-the-llm)
* [Simple AI Agent Flow](#llm-flow-with-tools-ai-agent)
* [Apply Tool Use Request from LLM](#apply-tool-use-request-from-the-llm)
* [Send Tool Response to the LLM](#tool-response)

### Visual improvements

We will use [rich library](https://github.com/Textualize/rich), and `rich-theme-manager` to make the output more readable, and supress warning messages.

In [1]:
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),      # Class names
            "repr.tag_name": "dim cyan",                        # Adjust tag names 
            "repr.call": "bright_yellow",                       # Function calls and other symbols
            "repr.str": "bright_green",                         # String representation
            "repr.number": "bright_red",                        # Numbers
            "repr.none": "dim white",                           # None
            "repr.attrib_name": Style(color="#e87d3e", bold=True),    # Attribute names
            "repr.attrib_value": "bright_blue",                 # Attribute values
            "default": "bright_white on black"                  # Default text and background
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          # Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     # Adjust tag names 
            "repr.call": Style(color="#ffff00", bold=True),         # Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          # String representation
            "repr.number": Style(color="#ff6347", bold=True),       # Numbers
            "repr.none": Style(color="#808080", bold=True),         # None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  # Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), # Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   # Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)

 Theme  Description       Tags  Path               
 dark   Dark mode theme   dark  themes/dark.theme  
 light  Light mode theme        themes/light.theme

                                      Theme: dark - themes/dark.theme                                      
┌───────────────────┬───────────────┬───────┬─────────┬─────────┬────────────────┬────────────────────────┐
│ style             │ color         │ color │ bgcolor │ bgcolor │ attributes     │ example                │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ default           │ bright_white  │ █████ │ black   │ █████   │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_name  │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_value │ bright_blue   │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.call         │ bright_yellow │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.none         │ white         │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.number       │ bright_red    │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.own          │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.str          │ bright_green  │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.tag_name     │ cyan          │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
└───────────────────┴───────────────┴───────┴─────────┴─────────┴────────────────┴────────────────────────┘
┌─ attributes legend ──────────────────────────────────────────────────────────────────┐
│  b: bold, d: dim, i: italic, u: underline, U: double underline, B: blink, 2: blink2  │
│  r: reverse, c: conceal, s: strike, f: frame, e: encircle, o: overline, L: Link      │
└──────────────────────────────────────────────────────────────────────────────────────┘

In [2]:
from rich.console import Console

dark = theme_manager.get("dark")
# Create a console with the dark theme
console = Console(theme=dark)


In [3]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Why LLM Hallucinate

We can ask a simple question our favorite LLM:

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
user_prompt = "What day is today?"

In [6]:
# Now time to connect to the large language model
from openai import OpenAI

openai_client = OpenAI()
completion = openai_client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "system", 
            "content": "You are chatbot, who is helping people with answers to their questions."
        },
        {
            "role": "user", 
            "content": 
                [
                    {"type": "text", "text": user_prompt},
                ],
        },
    ]
)

In [7]:
from rich.panel import Panel
from rich.text import Text

response_text = Text(completion.choices[0].message.content)
styled_panel = Panel(
    response_text,
    title=f"{user_prompt}",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

╭────────────────────────────────────────────── What day is today? ───────────────────────────────────────────────╮
│                                                                                                                 │
│ As an AI, I don't have real-time capabilities and cannot provide the current date. Please check the date on     │
│ your device.                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The LLM can't really help us.

## Create a simple tool in Python

If the naiive approach doesn't work, let's build an AI agent that will help us solve such tasks. First we need a `Tool` that can solve Sudoku puzzles. We can write it ourselves or use LLM for it.

In [8]:
code_request_prompt = "Write a Python function that returns today’s day of the week, such as ‘Sunday’ and ‘Monday’"

In [9]:
import anthropic

anthropic_client = anthropic.Anthropic()


In [10]:
response = anthropic_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=8192,
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": code_request_prompt,
                    }
                ]
            }
        ],
    )

In [11]:
console.print(response)

Message(
    id='msg_01JiLPRuAt33D6U3ZE49VnAm',
    content=[
        TextBlock(
            text='Here are a few ways to get today\'s day of the week in Python:\n\n```python\n# Method 1: Using 
datetime\nfrom datetime import datetime\n\ndef get_day_of_week():\n    return datetime.now().strftime(\'%A\')\n\n# 
Method 2: Using date from datetime\nfrom datetime import date\n\ndef get_day_of_week2():\n    return 
date.today().strftime(\'%A\')\n\n# Method 3: Using calendar\nimport calendar\nfrom datetime import datetime\n\ndef 
get_day_of_week3():\n    return calendar.day_name[datetime.now().weekday()]\n\n# Example 
usage:\nprint(get_day_of_week())   # e.g., "Wednesday"\nprint(get_day_of_week2())  # e.g., 
"Wednesday"\nprint(get_day_of_week3())  # e.g., "Wednesday"\n```\n\nEach of these methods will return the current 
day of the week as a string (like "Monday", "Tuesday", etc.).\n\nThe first two methods use strftime() with the 
\'%A\' format code, which gives the full weekday name. The third method uses the calendar module\'s day_name list 
indexed by the weekday number (0-6, where 0 is Monday).\n\nA simple one-liner version would be:\n\n```python\nfrom 
datetime import datetime\n\ndef get_day_of_week():\n    return datetime.now().strftime(\'%A\')\n```\n\nThis is 
probably the most straightforward and commonly used approach. The function will return the full name of the current
day of the week.\n\nIf you want to get the abbreviated day name (like "Sun", "Mon"), you can use \'%a\' instead of 
\'%A\':\n\n```python\nfrom datetime import datetime\n\ndef get_short_day_of_week():\n    return 
datetime.now().strftime(\'%a\')\n```\n\nUsage example:\n```python\nprint(get_day_of_week())        # e.g., 
"Wednesday"\nprint(get_short_day_of_week())  # e.g., "Wed"\n```\n\nThe function will automatically use your 
system\'s locale settings to determine the language of the day names. If you want to ensure English names 
regardless of the system\'s locale, you can set the locale explicitly:\n\n```python\nimport locale\nfrom datetime 
import datetime\n\ndef get_day_of_week():\n    # Set locale to English\n    locale.setlocale(locale.LC_TIME, 
\'en_US.UTF-8\')\n    return datetime.now().strftime(\'%A\')\n```\n\nChoose the method that best suits your needs. 
The first simple version using datetime is usually sufficient for most purposes.',
            type='text'
        )
    ],
    model='claude-3-5-sonnet-20241022',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=Usage(input_tokens=31, output_tokens=602)
)

In [12]:
def extract_code_from_llm_response(response_text):
    """
    Extract code between ```python and ``` markers from LLM response text
    
    Args:
        response_text (str): The text response from the LLM containing code blocks
        
    Returns:
        str: The extracted code, or None if no code block is found
    """
    code_start = response_text.find("```python")
    if code_start == -1:
        code_start = response_text.find("```")
    if code_start != -1:
        code_start = response_text.find("\n", code_start) + 1
        code_end = response_text.find("```", code_start)
        if code_end != -1:
            return response_text[code_start:code_end].strip()
    return None

In [13]:
# Extract code from the LLM response
extracted_code = extract_code_from_llm_response(response.content[0].text)
if extracted_code:
    print(extracted_code)

# Method 1: Using datetime
from datetime import datetime

def get_day_of_week():
    return datetime.now().strftime('%A')

# Method 2: Using date from datetime
from datetime import date

def get_day_of_week2():
    return date.today().strftime('%A')

# Method 3: Using calendar
import calendar
from datetime import datetime

def get_day_of_week3():
    return calendar.day_name[datetime.now().weekday()]

# Example usage:
print(get_day_of_week())   # e.g., "Wednesday"
print(get_day_of_week2())  # e.g., "Wednesday"
print(get_day_of_week3())  # e.g., "Wednesday"


## Tool Definition for the LLM

To allow the LLM to use the tool we need to provide a detailed description of the tool.

In [14]:
tools = [
    {
        "name": "get_today_day",
        "description": "Get the day of the week name of today.",
        "input_schema": {
            "type": "object",
            "properties": {}
        }
    }
]

In [15]:
user_initial_message = {
    "role": "user", 
    "content": [
        {
            "type": "text",
            "text": user_prompt,
        }
    ]
}

In [16]:
messages = []
messages.append(user_initial_message)

## LLM Flow with Tools (=AI Agent)

We will ask the LLM again, but this time we will provide it with a tool.

In [17]:
response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=8192,
    messages=messages,
    tools=tools
)

In [18]:
console.print(response)

Message(
    id='msg_01QPXoxwfHjjvuACNYFDbfP1',
    content=[
        TextBlock(
            text="I'll help you find out what day of the week it is today using the get_today_day function.",
            type='text'
        ),
        ToolUseBlock(id='toolu_01DfXSZRTYwZroY5iLsQHSXr', input={}, name='get_today_day', type='tool_use')
    ],
    model='claude-3-5-sonnet-20241022',
    role='assistant',
    stop_reason='tool_use',
    stop_sequence=None,
    type='message',
    usage=Usage(input_tokens=370, output_tokens=62)
)

### Apply Tool Use request from the LLM

We will append the reply from the LLM to the list of messages that includes the initial user questions. We will need it the list of the messages for the next time we call the LLM.

In [19]:
messages.append({"role": "assistant", "content": response.content})

In [20]:
if response.stop_reason == "tool_use":
    tool_use = next(block for block in response.content if block.type == "tool_use")
    tool_name = tool_use.name
    tool_input = tool_use.input
    tool_use_id = tool_use.id

    print(f"\nTool Used: {tool_name}")
    print("Tool Input:")
    print(tool_input)

    # code_interpreter_results = process_tool_call(e2b_code_interpreter, tool_name, tool_input)



Tool Used: get_today_day
Tool Input:
{}


We will execute the tool per the LLM request here to understand the flow.

In [27]:
from datetime import datetime

def get_day_of_week():
    return datetime.now().strftime('%A')
    
get_day_of_week()

'Friday'

In [22]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": get_day_of_week()
        }
    ]
}

### Tool Response

We will append the tool response to the messages list to be sent to the LLM.

In [23]:
messages.append(tool_response)

In [24]:
response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=8192,
    messages=messages,
    tools=tools
)

In [25]:
console.print(response)

Message(
    id='msg_01MvxhPkydKPGukbN1M2JWc4',
    content=[TextBlock(text='Today is Friday.', type='text')],
    model='claude-3-5-sonnet-20241022',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=Usage(input_tokens=444, output_tokens=8)
)

And we got the response to our `What day is today?` question from the LLM without hallucinations.

In [26]:
response_text = Text(response.content[0].text)
styled_panel = Panel(
    response_text,
    title=f"{user_prompt}",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

╭─ What day is today? ─╮
│                      │
│ Today is Friday.     │
│                      │
╰──────────────────────╯